In [1]:
# 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB
# и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.

# 2. Написать функцию, которая производит поиск и выводит на экран вакансии 
# с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты). 
# Для тех, кто выполнил задание с Росконтролем - напишите запрос 
# для поиска продуктов с рейтингом не ниже введенного или качеством не ниже введенного 
# (то есть цифра вводится одна, а запрос проверяет оба поля)

In [6]:
!pip install pymongo

In [60]:
import requests
from bs4 import BeautifulSoup as bs
from pymongo import MongoClient
import pprint
#import bson
#from bson.raw_bson import RawBSONDocument

import warnings
warnings.filterwarnings('ignore')

In [61]:
import pymongo
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
db = myclient['vacancies']
def connection_to_db():
    #Возвращение коннектора к базе данных Mongo
    hh_vacancies = db.hh_vacancies
    return db.vacancies

def get_hh_content(target):
    #Запрос к hh и возвращение контента
    return requests.get(f'https://api.hh.ru/vacancies={target}').text

def parse_hh_content(content):
    #Парсит содержание hh
    
    items = []

    soup = bs(content, 'html.parser')
    divs = soup.find_all('div', 'vacancy-serp-item')

    for div in divs:
        category = div.find('div', attrs={'class': 'data'}).content[1].content[0].strip()
        name = div.find('a', attrs={'class': 'item-description-title-link'}).text.strip()
        salary = int(div.find('span', attrs={'class': 'salary'})['content'])

        item = {
            'category': category,
            'name': name,
            'salary': salary
        }
        
        items.append(item)
    
    return items
        
def save_content_to_mongo(items):
    #Кладет контент в MongoDB
    db.hh_vacancies = connection_to_db()
    db.hh_vacancies.insert_many(items)

    #docs = [{'_id':1}, RawBSONDocument(bson.BSON.encode({'_id':2}))]
    #docs
    #[{'_id': 1}, RawBSONDocument('\x0e\x00\x00\x00\x10_id\x00\x02\x00\x00\x00\x00', codec_options=CodecOptions(document_class=<class 'bson.raw_bson.RawBSONDocument'>, tz_aware=False, uuid_representation=PYTHON_LEGACY, unicode_decode_error_handler='strict', tzinfo=None))]
    #client.test.test.insert_many(items)
    #<pymongo.results.InsertManyResult object at 0x106c32758>
    #list(client.test.test.find())
    #[{u'_id': 1}, {u'_id': 2}]

In [41]:
    # for div in divs:
        # category = div.find('div', attrs={'class': 'vacancy-serp__vacancy vacancy-serp__vacancy_standard_plus'}).content[1].content[0].strip()
        # name = div.find('a', attrs={'class': 'bloko-link'}).text.strip()
        # salary = int(div.find('span', attrs={'class': 'bloko-header-section3'})['vacancies'])

In [64]:
# Скачивает контент
content = get_hh_vacancies('teacher')

# Парсит контент
items = parse_hh_content(content)

#Удаляет старый контент
db.hh_vacancies = connection_to_db()
db.hh_vacancies.drop()

#Сохраняет контент в базу
save_content_to_mongo(items)

TypeError: documents must be a non-empty list

In [65]:
db.hh_vacancies.count()

TypeError: 'Collection' object is not callable. If you meant to call the 'count' method on a 'Collection' object it is failing because no such method exists.

In [66]:
def get_item_with_salary_less(salary_limit=None):
    #Выводит вакансии ниже указанной суммы
    vacancies_db = connection_to_db()
    
    if not salary_limit:
        salary_limit = input("Insert max salary: ")
    
    print('\n')
    for p in vacancies_db.find({"$and": [{"salary": {"$gt": 0}}, {"salary": {"$lt": int(salary_limit)}}]}).sort("salary"):
        pprint.pprint(p)

In [67]:
get_item_with_salary_less()

Insert max salary: 300


